In this example kernel, we try to demonstrate the LP for the Sudoku game. To study the problem 

$$\min_{X} \|X\|_{L^1} $$
subject to equality constraint $AX = B$.

In [1]:
import os
print(os.listdir("input"))

['.DS_Store', 'large1.csv', 'large2.csv', 'small1.csv', 'small2.csv']


In [37]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy.sparse as scs # sparse matrix construction 
import scipy.linalg as scl # linear algebra algorithms
import scipy.optimize as sco # for minimization use
import matplotlib.pylab as plt # for visualization
from collections import defaultdict
import math
def fixed_constraints(N=9):
    rowC = np.zeros(N)
    rowC[0] =1
    rowR = np.zeros(N)
    rowR[0] =1
    row = scl.toeplitz(rowC, rowR)
    ROW = np.kron(row, np.kron(np.ones((1,N)), np.eye(N)))
    
    colR = np.kron(np.ones((1,N)), rowC)
    col  = scl.toeplitz(rowC, colR)
    COL  = np.kron(col, np.eye(N))
    
    M = int(np.sqrt(N))
    boxC = np.zeros(M)
    boxC[0]=1
    boxR = np.kron(np.ones((1, M)), boxC) 
    box = scl.toeplitz(boxC, boxR)
    box = np.kron(np.eye(M), box)
    BOX = np.kron(box, np.block([np.eye(N), np.eye(N) ,np.eye(N)]))
    
    cell = np.eye(N**2)
    CELL = np.kron(cell, np.ones((1,N)))
    
    return scs.csr_matrix(np.block([[ROW],[COL],[BOX],[CELL]]))
#print(fixed_constraints(N=9))

# For the constraint from clues, we extract the nonzeros from the quiz string.
def clue_constraint(input_quiz, N=9):
    m = np.reshape([int(c) for c in input_quiz], (N,N))
    #print(m)
    r, c = np.where(m.T)
    v = np.array([m[c[d],r[d]] for d in range(len(r))])
    
    table = N * c + r
    table = np.block([[table],[v-1]])
    #print(table)
    # it is faster to use lil_matrix when changing the sparse structure.
    CLUE = scs.lil_matrix((len(table.T), N**3))

    for i in range(len(table.T)):
        CLUE[i,table[0,i]*N + table[1,i]] = 1
    # change back to csr_matrix.
    CLUE = CLUE.tocsr() 
    
    return CLUE
#print(clue_constraint( '000000000000000127009002530000000000004000981080754000000000000060089003473105000', N=9))

In [24]:
def clean(x):
    if x > 9:
        x = 0
    return x

In [8]:
# def forward_checking(input_quiz , N=9):
#     m = np.reshape([int(c) for c in input_quiz], (N,N))
#     #r, c = np.where(m.T)
#     #v = np.array([m[c[d],r[d]] for d in range(len(r))])
    
#     for i in range(N):
#         for j in range(N):
#             if m[i][j] == 0:
#                 m[i][j] = 123456789
#                 for k in m[i]:
#                     if k < 10:
#                         s = str(m[i][j]).replace(str(k), '0')
#                         m[i][j] = int(s)
#                 for k in m[:,j]:
#                     if k < 10:
#                         s = str(m[i][j]).replace(str(k), '0')
#                         m[i][j] = int(s)
#                 dd = my_block(m) #block dict
#                 ll = dd[(math.ceil(i/N), math.ceil(j/N))]
#                 #for k in ll:         #eliminate block
#                     #if k < 10:
#                         #s = str(m[i][j]).replace(str(k), '0')
#                         #m[i][j] = int(s)
                
#     return m

def forward_checking(input_quiz , N=9):
#     print(input_quiz)
    m = np.reshape([int(c) for c in input_quiz],(N,N))
    
    for i in range(N):
        for j in range(N):
           # print(i, j,m[i][j])
            if m[i][j] == 0:
                m[i][j] = 123456789
               
                for k in m[i]:
                   # print(m)
                   # print("k",k)
                    if k < 10 and k != 0:
                        string = str(m[i][j])
                        if str(k) in string:
                            if len(string) > 1:
                                string = string.replace(str(k), '')
                       # print("yes",string)
                        m[i][j] = int(string)
                        
                        
                for k in m[:,j]:
                   # print("k",k)
                    if k < 10 and k != 0:
                        string = str(m[i][j])
                        if str(k) in string and len(string) > 1:
                            string = string.replace(str(k), '')
                       # print("no",string)
                        m[i][j] = int(string)
    for i in range(N):
        for j in range(N):
            if m[i][j] > 9:
                m[i][j] = 0
                
    return m



def my_block(mygb, N=9):
    dd = defaultdict(list)
    for i in range(len(mygb)):
        for j in range(len(mygb[i])):
            key = (math.ceil(i/N), math.ceil(j/N))
            dd[key].append(mygb[i][j])
    return dd

print(forward_checking( '000000000000000127009002530000000000004000981080754000000000000060089003473105000', N=9))


[[123056789 123450009 120056780  23456089 123406709 103006780  23406780
  100456709  20456089]
 [  3056089   3450009     56080   3456089   3406009   3006080         1
          2         7]
 [100006780 100400000         9    406080 100406700         2         5
          3    406080]
 [123056789 123450009 120056780  23456089 123406709 103006780  23406780
  100456709  20456089]
 [ 23056700  23050000         4  23056000  23006700   3006700         9
          8         1]
 [123006009         8 120006000         7         5         4  23006000
  100006009  20006009]
 [123056789 123450009 120056780  23456089 123406709 103006780  23406780
  100456709  20456089]
 [120050700         6 120050700  20450000         8         9  20400700
  100450700         3]
 [        4         7         3         1  20006009         5  20006080
       6009  20006089]]


In [44]:
small2 = pd.read_csv('input/small2.csv')
small1 = pd.read_csv('input/small1.csv')
large2 = pd.read_csv('input/large2.csv')
large1 = pd.read_csv('input/large1.csv')


In [45]:
test1 = small1['quizzes'][0]
sol1 = small1['solutions'][0]
test2 = small2['quizzes'][0]
sol2 = small2['solutions'][0]

In [51]:
N = 24
n = 0
for i in range(N):
    sol = small1['solutions'][i]
    test = small1['quizzes'][i]
    sol_mat = np.reshape([int(c) for c in sol], (9,9))
    test_mat = np.reshape([int(c) for c in test], (9,9))
    result = forward_checking(test)
    clean_result = np.vectorize(clean)(result)
    if np.any(clean_result!=test_mat):
#         print(i)
#         print(sol_mat)
#         print(test_mat)
#         print(clean_result)
#         print()
        n +=1
print('sucess percentage:', n/N)

sucess percentage: 0.125


Use ``scipy.optmize`` to find the solution. We test the method on ``large1`` with random seed 42 to generate the 1000 samples.

In [ ]:
import time

# We test the following algoritm on small data set.
data = pd.read_csv("../input/small2.csv") 

corr_cnt = 0
start = time.time()

random_seed = 42
np.random.seed(random_seed)

if len(data) > 1000:
    samples = np.random.choice(len(data), 1000)
else:
    samples = range(len(data))

for i in range(len(samples)):
    quiz = data["quizzes"][samples[i]]
    solu = data["solutions"][samples[i]]
    A0 = fixed_constraints()
    A1 = clue_constraint(quiz)

    # Formulate the matrix A and vector B (B is all ones).
    A = scs.vstack((A0,A1))
    A = A.toarray()
    B = np.ones(A.shape[0])


    # Because rank defficiency. We need to extract effective rank.
    u, s, vh = np.linalg.svd(A, full_matrices=False)
    K = np.sum(s > 1e-12)
    S = np.block([np.diag(s[:K]), np.zeros((K, A.shape[0]-K))])
    A = S@vh
    B = u.T@B
    B = B[:K]

    c = np.block([ np.ones(A.shape[1]), np.ones(A.shape[1]) ])
    G = np.block([[-np.eye(A.shape[1]), np.zeros((A.shape[1], A.shape[1]))],\
                         [np.zeros((A.shape[1], A.shape[1])), -np.eye(A.shape[1])]])
    h = np.zeros(A.shape[1]*2)
    H = np.block([A, -A])
    b = B

    ret = sco.linprog(c, G, h, H, b, method='interior-point', options={'tol':1e-6})
    x = ret.x[:A.shape[1]] - ret.x[A.shape[1]:]

    
    z = np.reshape(x, (81, 9))
    if np.linalg.norm(np.reshape(np.array([np.argmax(d)+1 for d in z]), (9,9) ) \
                      - np.reshape([int(c) for c in solu], (9,9)), np.inf) >0:
        pass
    else:
        #print("CORRECT")
        corr_cnt += 1

    if (i+1) % 20 == 0:
        end = time.time()
        print("Aver Time: {t:6.2f} secs. Success rate: {corr} / {all} ".format(t=(end-start)/(i+1), corr=corr_cnt, all=i+1) )

end = time.time()
print("Aver Time: {t:6.2f} secs. Success rate: {corr} / {all} ".format(t=(end-start)/(i+1), corr=corr_cnt, all=i+1) )